# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDRegressor, LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix

from sklearn.datasets import load_digits

df = pd.read_csv('data/bank-additional-full.csv', sep = ';').dropna()
df.info()
#df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [ ]:
# Check for empty vaules
# df.isnull().sum().sort_values().plot(kind = 'bar')
# plt.show();

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [2]:
categorical = ['age','job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
numerical = df.drop(columns=categorical).columns
numerical = numerical.drop(['y'])

print(numerical)
print(categorical)
#print(df[categorical])

Index(['duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


In [3]:
# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical),
        ('cat', OneHotEncoder(), categorical)
    ])

In [ ]:
sns.pairplot(df,hue='y')

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [4]:
X = df.drop('y', axis=1)
y = df['y']#.astype(int)

X_scaled = preprocessor.fit_transform(X)
print( X_scaled[5] )

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 42)

  (0, 0)	-0.23251279193519303
  (0, 1)	-0.5659219741930245
  (0, 2)	0.1954139001271294
  (0, 3)	-0.34949428419757467
  (0, 4)	0.6480922665892298
  (0, 5)	0.7227224704883503
  (0, 6)	0.8864465616355264
  (0, 7)	0.7124598790361739
  (0, 8)	0.3316799069141756
  (0, 37)	1.0
  (0, 94)	1.0
  (0, 100)	1.0
  (0, 105)	1.0
  (0, 112)	1.0
  (0, 114)	1.0
  (0, 117)	1.0
  (0, 121)	1.0
  (0, 128)	1.0
  (0, 133)	1.0
  (0, 138)	1.0


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
# Linear Regression

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [5]:
# GridSearch the Log Reg penalty parameter
# Best choice = None

# logreg_params = {'penalty': ['l1', 'l2', 'elasticnet', None]}
#logreg_grid = GridSearchCV(LogisticRegression(),param_grid=logreg_params)
#logreg_grid.fit(X_train,y_train)

#best_penalty = list(logreg_grid.best_params_.values())[0]
#best_acc = logreg_grid.score(X_test, y_test)

#print(best_acc)
#print(best_penalty)

#df1 = pd.DataFrame(logreg_grid.cv_results_)
#time1 = logreg_grid.cv_results_['mean_fit_time'].sum()
#index1 = np.argmax(df1['mean_test_score'])

#print(f'Compute time: {time1}')
#print(f'Parameters of Max Test Score: {df1['params'][index1]}')
#print(f'Max Test Score: {df1['mean_test_score'][index1]}')

start = time.time()
lgr = LogisticRegression(multi_class='multinomial').fit(X_train, y_train)
end = time.time()

### Problem 9: Score the Model

What is the accuracy of your model?

In [6]:
lgr_train_score = lgr.score(X_train,y_train)
lgr_test_score = lgr.score(X_test,y_test)
lgr_elapsed_time = end - start

print(f'Training Accuracy Score: {lgr_train_score}')
print(f'Test Accuracy Score: {lgr_test_score}')
print(f'Training Time (s): {lgr_elapsed_time}')

Training Accuracy Score: 0.9121750671716681
Test Accuracy Score: 0.9119160920656502
Training Time (s): 0.46871495246887207


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

## k Nearest Neighbors

In [7]:
start1 = time.time()
knn_final = KNeighborsClassifier().fit(X_train, y_train)
end1 = time.time()

knn_train_score = knn_final.score(X_train, y_train)
knn_test_score = knn_final.score(X_test,y_test)
knn_elapsed_time = end1 - start1

print(f'Training Accuracy Score: {knn_train_score}')
print(f'Test Accuracy Score: {knn_test_score}')
print(f'Training Time (s): {knn_elapsed_time}')

Training Accuracy Score: 0.9283286394095368
Test Accuracy Score: 0.9024958725842478
Training Time (s): 0.021941661834716797


## Decision Tree

In [8]:
start = time.time()
decTree_final = DecisionTreeClassifier().fit(X_train, y_train)
end = time.time()

decTree_train_score = decTree_final.score(X_train, y_train)
decTree_test_score = decTree_final.score(X_test,y_test)
decTree_elapsed_time = end - start

print(f'Training Accuracy Score: {decTree_train_score}')
print(f'Test Accuracy Score: {decTree_test_score}')
print(f'Training Time (s): {decTree_elapsed_time}')

Training Accuracy Score: 1.0
Test Accuracy Score: 0.8921044964552782
Training Time (s): 1.0153169631958008


## SVM Model

In [9]:
start2 = time.time()
svc_final = SVC().fit(X_train, y_train)
end2 = time.time()

svc_train_score = svc_final.score(X_train, y_train)
svc_test_score = svc_final.score(X_test,y_test)
svc_elapsed_time = end2 - start2

print(f'Training Accuracy Score: {svc_train_score}')
print(f'Test Accuracy Score: {svc_test_score}')
print(f'Training Time (s): {svc_elapsed_time}')

Training Accuracy Score: 0.9241202939367453
Test Accuracy Score: 0.9122074390599204
Training Time (s): 22.934684991836548


## Results

In [10]:
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'Decision Tree', 'SVM'],
    'Fit Time': [lgr_elapsed_time, knn_elapsed_time, decTree_elapsed_time, svc_elapsed_time],
    'Training Score': [lgr_train_score, knn_train_score, decTree_train_score, svc_train_score],
    'Test Score': [lgr_test_score, knn_test_score, decTree_test_score, svc_test_score]
})

print(results)

                 Model   Fit Time  Training Score  Test Score
0  Logistic Regression   0.468715        0.912175    0.911916
1                  KNN   0.021942        0.928329    0.902496
2        Decision Tree   1.015317        1.000000    0.892104
3                  SVM  22.934685        0.924120    0.912207


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

### Grid Searches for KNN, Decision Tree, and SVC

#### KNN

In [14]:
params = {'n_neighbors': list(range(1, 50, 2))}

knn_grid = RandomizedSearchCV(KNeighborsClassifier(),params)
#knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid=params)

knn_grid.fit(X_train, y_train)
best_k = list(knn_grid.best_params_.values())[0]
best_acc = knn_grid.score(X_test, y_test)

print(best_acc)
print(best_k)

df1 = pd.DataFrame(knn_grid.cv_results_)
time1 = knn_grid.cv_results_['mean_fit_time'].sum()
index1 = np.argmax(df1['mean_test_score'])

print(f'Compute time: {time1}')
print(f'Parameters of Max Test Score: {df1['params'][index1]}')
print(f'Max Test Score: {df1['mean_test_score'][index1]}')

0.9065747305040303
41
Compute time: 0.22084555625915528
Parameters of Max Test Score: {'n_neighbors': 41}
Max Test Score: 0.906995592953105


#### Decision Tree

In [15]:
tree_params = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}

dtc_grid = RandomizedSearchCV(DecisionTreeClassifier(), tree_params)
#dtc_grid = GridSearchCV(DecisionTreeClassifier(), param_grid=tree_params)

dtc_grid.fit(X_train, y_train)
best_criterion = dtc_grid.best_params_['criterion']
best_depth = dtc_grid.best_params_['max_depth']

print(f'Best Criterion Type: {best_criterion}')
print(f'Best Max Depth: {best_depth}')

df1 = pd.DataFrame(dtc_grid.cv_results_)
time1 = dtc_grid.cv_results_['mean_fit_time'].sum()
index1 = np.argmax(df1['mean_test_score'])

print(f'Compute time: {time1}')
print(f'Parameters of Max Test Score: {df1['params'][index1]}')
print(f'Max Test Score: {df1['mean_test_score'][index1]}')

Best Criterion Type: entropy
Best Max Depth: 7
Compute time: 4.757117748260498
Parameters of Max Test Score: {'max_depth': 7, 'criterion': 'entropy'}
Max Test Score: 0.91282269003854


#### SVM

In [16]:
svm_params = {'kernel': ['rbf', 'poly', 'linear', 'sigmoid'],
         'gamma': [0.1, 1.0, 10.0, 50.0, 100.0],}


svm_grid = RandomizedSearchCV(SVC(), svm_params)
#svm_grid = GridSearchCV(SVC(), param_grid=svm_params)
svm_grid.fit(X_train, y_train)

best_kernel = svm_grid.best_params_['kernel']
best_gamma = svm_grid.best_params_['gamma']

print(f'Best Kernel Type: {best_kernel}')
print(f'Best Gamma: {best_gamma}')

df1 = pd.DataFrame(svm_grid.cv_results_)
time1 = svm_grid.cv_results_['mean_fit_time'].sum()
index1 = np.argmax(df1['mean_test_score'])

print(f'Compute time: {time1}')
print(f'Parameters of Max Test Score: {df1['params'][index1]}')
print(f'Max Test Score: {df1['mean_test_score'][index1]}')

Best Kernel Type: linear
Best Gamma: 10.0
Compute time: 1076.2005663394928
Parameters of Max Test Score: {'kernel': 'linear', 'gamma': 10.0}
Max Test Score: 0.9036936687202358


##### Questions